In [72]:
import json
with open("scopus.config") as handle:
    config = json.load(handle)

In [35]:
import pandas as pd
xls = pd.ExcelFile(config['file'])

In [36]:
df = pd.read_excel(xls, 
                   sheet_name=config['sheet_name'], 
                   header=config['header']
                   )

Imported everything. Marking "x" in column "703" (ID of IT category) indicates the IT tag of object.

In [37]:
refined = df[df[config['category']] == 'x'][df['Punkty'] > config['points_treshold']]
search_list = pd.concat([refined[e] for e in config['extract']]).dropna().unique()
assert len(search_list) == len(refined)

/tmp/ipykernel_10591/2874088279.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  refined = df[df[config['category']] == 'x'][df['Punkty'] > config['points_treshold']]


In [76]:
def make_query(elements, keyword, conjugator):
    answer = keyword + "("
    for e in elements:
        answer += f"{e} {conjugator} "
    answer = answer[:-(len(conjugator) + 1)]
    answer += ")"
    return answer

In [77]:
with open('scopus.answer', "w") as handle:
    handle.write(make_query(search_list, config['keyword'], config['conjugator']))